In [22]:
from cgi import print_environ
import requests
import base64
import csv
import pandas as pd
from datetime import datetime
import os

# TODO: Add a Class to handle the API calls

# --------------------------EXTRACT STAGE-----------------------------------
# Replace these with your actual values
subdomain = "scantrust.zendesk.com"  # Replace with your Zendesk subdomain
# Replace with the email associated with your token  "nicolas.garces@scantrust.com"
email_address = "nicolas.garces@scantrust.com"
api_token = "O0UKFHZcy9BPNVyBUhYWvlRLcMwVUIOx1BWruGnE"  # Replace with your API token

# Construct the API URL
# url = f"https://{subdomain}/api/v2/users.json?page=1"
url = f"https://{subdomain}/api/v2/users"

temp_dir = "temp_csvs"
os.makedirs(temp_dir, exist_ok=True)

# Transofrm the data
# Convert API response data to a DataFrame


def transform_data(users):
    # convert API response data to a DataFrame
    df = pd.DataFrame(users)

    # fill missing values with a default value
    df = df.fillna("N/A")

    # Add a column with the current date
    df["processed_date"] = datetime.today().strftime('%Y-%m-%d')

    # convert 'created_at' and 'updated_at' columns to datetime
    df["created_at"] = pd.to_datetime(
        df["created_at"], format='%Y-%m-%dT%H:%M:%SZ')
    df["updated_at"] = pd.to_datetime(
        df["updated_at"], format='%Y-%m-%dT%H:%M:%SZ')

    return df


page_num = 1  # Start with the first page

while url:
    print(url)

    # Encode the email and token for the Authorization header
    auth_string = f"{email_address}/token:{api_token}"
    base64_auth_string = base64.b64encode(
        auth_string.encode('utf-8')).decode('utf-8')

    headers = {
        "Authorization": f"Basic {base64_auth_string}",
        "Content-Type": "application/json"
    }

    # Send the GET request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        user_data = response.json()
        users = user_data["users"]

        cleaned_data = transform_data(users)
        csv_path = os.path.join(temp_dir, f"zendesk_user_data_{page_num}.csv")
        cleaned_data.to_csv(csv_path, index=False)

        url = user_data["next_page"]
        page_num += 1

    else:
        print(f"Request failed with status code {response.status_code}")
        break

# concatenate all CSVs into one Dataframe
csv_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir)]
df_list = [pd.read_csv(f) for f in csv_files]
all_data = pd.concat(df_list, ignore_index=True)

# Export the data to a CSV file with the current date
csv_file_path = f"zendesk_user_data_{datetime.today().strftime('%Y-%m-%d')}.csv"
all_data.to_csv(csv_file_path, index=False)

# remove the temporary CSV files
for f in csv_files:
    os.remove(f)
os.rmdir(temp_dir)

PermissionError: [Errno 13] Permission denied: 'zendesk_user_data.csv'